# <center> <font color = 'rebeccapurple'>Stochastic Watershed Model: Generating Stochastic Streamflows for the Shasta River Under 0 Degrees of Warming (Baseline Scenario)</font> </center>

This Jupyter Notebook contains code for a stochastic watershed model (SWM) that can be used to develop an ensemble of stochastic streamflows for a single site. This code was used to produce the results found in **Shabestanipour et al. (2024)** for the Shasta River basin. With modifications to the input data and the specific ARMA model used (more details below), this code could be applied to other non-regulated river basins. 

This notebook is split into two parts: 1. SWM (generating stochastic streamflows) and 2. Diagnostics (assessing stochastic model performance).

***

**Computational Requirements**: We used Jupyter version 4.7.1 and Python version 3.8.8 to run the below code. Lower versions may result in error messages.

The code is somewhat computationally-intesnive. We found that allocating 32GB of memory allowed the code to run in full without crashing the kernel. 

The code blocks that will take the longest are generating the stochastic residuals in Part 1 and generating the figure inputs in Part 2 (depending on how many stochastic realizations you do). These should each take approx. 10 min maximum. Most other code blocks run very quickly. The code to generate AIC/BIC for various ARIMA models to determine which to use may also take some time (by default this code block is commented out).

***

# <center> <font color = 'red'> Part 1: SWM </font> </center>

## <font color='green'> Import Statements </font>

Here we import SWM functions (developed by Ghazal Shabestanipour) as well as existing external Python packages used in the notebook. Be sure that the file **Func_Lib.py** is contained in the same directory as this Jupyter Notebook.
For existing Python packages, if you do not have a particular package installed with your version of Python, you can install it using pip or conda (if using anaconda). Instructions for doing so can be found here:<br>
https://packaging.python.org/en/latest/tutorials/installing-packages/ (pip)<br>
https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/installing-with-conda.html (conda)

In [24]:
# functions developed by Ghazal:
import Func_Lib_forAbby as func

# general packages
import numpy as np #numpy is a scientific computing package
import pandas as pd #pandas is a data analysis package
import math #math contains math functions
import time #functions in this package can be used to understand how long the code is taking to run
import os #functions related to operating system

# plotting packages
import matplotlib.pyplot as plt #matplotlib.pyplot is used for plotting
import seaborn as sns #seaborn is a plotting package
plt.style.use('seaborn-white') #this is the specific style of plotting used

# statistical analysis packages
from scipy import stats #scipy.stats are functions related to statistical analysis
from scipy.stats import norm #functions for normally distributed random variable 

# time series analysis packages
import statsmodels as sm
from statsmodels.graphics.tsaplots import plot_acf #function to plot autocorrelation function (acf) for time series analysis
from statsmodels.graphics.tsaplots import plot_pacf #function to plot partial autocorrelation function (pacf) for time series analysis
#from statsmodels.tsa.arima_model import ARIMA #import function for ARIMA (time series analysis)
from statsmodels.tsa.arima.model import ARIMA

## <font color='green'> Load Input Data </font>

The only input data required is a csv containing daily streamflow data (observed and modeled). The columns of the csv are $date$, observed streamflow $Qgage$, deterministic model of streamflow $Qmodel$, and the difference between Qgage and Qmodel for each date $diff$.

For the Squannacook, we have values from 10/01/1987 - 09/30/2018.

In [25]:
# read in the csv of streamflow data and assign to variable called "data"
data = pd.read_csv('SWM-data_cond-debias.csv') #columns are date, Qgage (observed), Qmodel (deterministic model), and diff (Qmodel - Qgage)

data['month'] = pd.DatetimeIndex(data['date']).month #create a DatetimeIndex column for month
data['date'] = pd.to_datetime(data['date']) #convert format of "date" column to Datetime (for time series analysis)

#convert units to m^3/s 
# mm/day * SHA area in mi2 * (1609.34^2 m^2/mi^2) * (10^-3 m/1 mm) * (1 day/24*3600 s)
area_mi2_SHA = 6665 #area for Shasta baisn in sq. mi. from Zach Brodeur
data['Qgage'] = data['Qgage']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
data['Qmodel'] = data['Qmodel']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)

# generating empirical log-ratio errors
data['lambda'] = np.log(data['Qmodel'] / data['Qgage']) #calculate the lambda = log(Qmodel/Qgage)
data['raw diff'] = data['Qmodel'] - data['Qgage']
data #display first five rows of data

,date,Qgage,Qmodel,month,lambda,raw diff
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714
...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763


In [26]:
#load in a "future" scenario into a new datafrae df_fut
df_fut = pd.read_csv('SWM-data_cond-debias_scenario-1.csv') #make sure this is one of the five we want, I just used a random one

#put the future scenario into the correct units of m3/s
df_fut['Qfut'] = df_fut['Qsim_cbias']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
df_fut['date'] = pd.to_datetime(df_fut['date'])
df_fut = df_fut[(df_fut['date'] > '1987-09-30') & (df_fut['date'] < '2018-10-01')]
df_fut = df_fut.reset_index()
df_fut['index']=np.arange(0, 11323)

#df_fut = df_fut.iloc[0:11323]

df_fut

,index,date,Qsim,Qsim_cbias,Qfut
0,0,1987-10-01,0.010082,0.422703,84.453473
1,1,1987-10-02,0.009904,0.421925,84.298002
2,2,1987-10-03,0.009714,0.421092,84.131500
3,3,1987-10-04,0.009538,0.420316,83.976600
4,4,1987-10-05,0.009348,0.419472,83.807879
...,...,...,...,...,...
11318,11318,2018-09-26,0.013803,0.392048,78.328773
11319,11319,2018-09-27,0.013377,0.390136,77.946769
11320,11320,2018-09-28,0.012988,0.388372,77.594314
11321,11321,2018-09-29,0.017419,0.407531,81.422180


## <font color='green'> Fit ARMA Model to Data </font>

Here we fit an AR4 model to our "lambdas" to get the model residuals and parameters.

In [27]:
p=4 #order of AR (autocorrelation with lag 4)
d=0 #order of I (no trend)
q=0 #order of MA (no moving average component)

mod = ARIMA(data['lambda'], order=(p,d,q)) #set up AR4 model using lambdas as input
res = mod.fit() #fit model

In [28]:
res.params #display the model parameters

const     0.023440
ar.L1     0.165057
ar.L2     0.187447
ar.L3     0.147261
ar.L4     0.137038
sigma2    0.067909
dtype: float64

Since we fit an AR4 model for this data, we have five parameters: a coefficient for each AR component (L1, L2, L3, and L4) and a constant term.

In [29]:
#create a column in data for the model residuals associated with each time step
data['Ar_res']=res.resid
data.head(5)

,date,Qgage,Qmodel,month,lambda,raw diff,Ar_res
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708,1.786596
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749,1.411201
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814,0.761985
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705,0.583921
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714,0.070037


## <font color = 'green'> Generate Random AR Parameter Sets </font>

We generate $m$ random AR parameter sets (called "Beta") to consider the uncertainty in the AR model parameters.

We use the Numpy function **random.multivariate_normal** to generate the parameter sets. This function samples the parameters from a multivariate normal distribution. The inputs to the function are the mean **(res.params)**, the covariance **(res.cov_params)**, and the number of parameter sets to generate. 

In [30]:
# m is the number of random AR parameter sets, Beta, generated for considering AR model uncertainty.
m = 100

# older version of ARIMA function
#Beta=np.random.multivariate_normal(np.array(res.params), np.array(res.cov_params()), m)

# updated Beta for using newer version of statsmodels ARIMA code
# Beta=np.random.multivariate_normal(np.array(res.params[:-1]), np.array(res.cov_params())[:-1,:-1], m)

# for reproducible Betas, use scipy.stats multivariate_normal function instead, setting random seed
Beta = stats.multivariate_normal(np.array(res.params[:-1]),np.array(res.cov_params())[:-1,:-1], seed=16)
Beta = Beta.rvs(m)

## <font color = 'green'> Generate Stochastic Residuals Using Simple Bootstrap </font>

There are 3 steps to generating the stochastic streamflow realizations:<br>

1. Generate stochastic residuals (use either simple bootstrap, conditional bootstrap, or k-Nearest Neighbors bootstrap)
2. Generate stochastic log errors by combining AR model uncertainty + stochastic residuals in AR4 model
3. Generate stochastic streamflow by transforming from log-ratios back to streamflows, including bias correction factor<br>

### <font color = 'green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 1: Generate Stochastic Residuals <green>

In [31]:
#now find the nearest Qmodel and residual associated with Qmodel for each flow value in Qfut
df_fut['Qmodel_nearest'] = 0 #initialize empty column for nearest Qmodel
df_fut['residual_nearest'] = 0 #initialize empty column for nearest Qmodel residual

for i in range(len(df_fut)): #loop through each row of df_fut
    #for each flow, calculate absolute difference of that flow and all flows in Qmodel
    #take absolute value and then find the index of the minimum absolute value
    #populate Qmodel_nearest column with data['Qmodel'] at this index
    #and residual_nearest column with data['Ar_res'] at this index
    df_fut['Qmodel_nearest'].iloc[i] = data['Qmodel'].iloc[abs(data['Qmodel'] - df_fut['Qfut'].iloc[i]).idxmin()]
    df_fut['residual_nearest'].iloc[i] = data['Ar_res'].iloc[abs(data['Qmodel'] - df_fut['Qfut'].iloc[i]).idxmin()]
    
#now use KNN bootstrapping as before, with Epsilon defined as the Qmodel_nearest and residual_nearest columns from df_fut
#basically exactly what we had before, with some data "repeats" to correspond to the new data points
#and in the correct order to match Qfut

n = 100 # n is the number of stochastic streamflow realizations generated for each Beta set
Epsilon = df_fut.filter(['residual_nearest','Qmodel_nearest'])
Epsilon = np.array(Epsilon) # turn dataframe into numpy array

E = [] #initialize empty list to store errors
for t in range(0,len(df_fut)): # loop through all time periods
    E.append(Epsilon - [0,Epsilon[t,1]])  # first column residual, second column diff between flow and all other flows

k = 750

Er = np.zeros((len(df_fut),k)) #initialize empty 2d array with dimensions time periods x k (# nearest neighbors)
for t in range(0,len(df_fut)): #loop through all time periods
    E[t][:,1] = np.abs(E[t][:,1]) #get absolute values of errors
    x = E[t][np.argsort(E[t][:, 1])] #sort errors by magnitude
    Er[t,:] = x[0:k,0]
    

R = np.zeros((len(df_fut),n*m)) #initialize empty 2d array with dimensions time periods x # realizations
for i in range(0,n*m): #loop through all realizations
    I = np.random.randint(low=0, high=k, size=len(df_fut)) #randomly identify indices
    R[:,i] = Er[tuple(np.arange(0,len(df_fut),1)),I] #sample from Er at random indices I
    
E = R #set E to equal R

/cluster/tufts/hpc/tools/anaconda/202105/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


One needs to experiment with different k values to find the appropriate k for each basin by going trough all the next steps of the SWM and diagnostic figures.  

### <font color='green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 2: Generate Stochastic Log Errors </font>

In [32]:
# l is a matrix of [len(data),m*n] representing stochastic log-ratio errors         
L = np.zeros((len(df_fut),m*n)) #initialize empty array to populate below

# for the first five time periods, we set values to be the deterministic model lambdas across all realizations
# this is because we do not have enough time periods in the past to populate the AR4 model stochastically
for i in range(0,m*n):
    L[0:5,i] = data['lambda'].iloc[0:5] 

for i in range(0,m):    
    B = Beta[i,:] #let B = random parameter set for realization i
    for t in range(5,len(df_fut)):
        # L(t) = B0 + B1*L[t-1] + B2*L[t-2] + B3*L[t-3] + B4*L[t-4] + E[t]
        L[t, i*n:((i+1)*n)] = B[0] + \
                              B[1] * L[t-1, i*n:((i+1)*n)] + \
                              B[2] * L[t-2, i*n:((i+1)*n)] + \
                              B[3] * L[t-3, i*n:((i+1)*n)] + \
                              B[4] * L[t-4, i*n:((i+1)*n)] + \
                              E[t, i*n:((i+1)*n)]

### <font color = 'green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 3: Generate Stochastic Streamflows </font>

In [33]:
# Q is a matrix of stochastically simulated streamflows of size [m*n, len(data)]
# Just like lambda = log(Qmodel/Qgage), we now set Q = Qmodel/e^L where
# L is stochastic lambdas and Q is stochastic streamflow
Q = np.array(df_fut['Qfut']) / np.exp(L.T)

# BCF is transformation bias correction factor 
# this address the bias introduced by moving to log space
BCF = 1 / np.exp(-np.mean(data['lambda']) + np.var(data['lambda'])/2)

Q = Q*BCF

In [34]:
print(BCF)

0.9807641078478005


In [35]:
display(pd.DataFrame(Q))

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,13.554817,11.230534,14.981784,15.615197,24.392023,39.726960,52.988130,51.933990,37.650044,43.614619,...,53.165617,55.084171,72.451372,81.321422,72.248276,74.090379,63.557573,58.086821,76.039594,84.917629
1,13.554817,11.230534,14.981784,15.615197,24.392023,30.025620,28.092256,33.139409,32.449148,22.103253,...,81.752544,128.999411,88.146602,120.520885,104.948489,99.009263,75.639922,102.063779,100.251621,37.049554
2,13.554817,11.230534,14.981784,15.615197,24.392023,31.887895,41.971683,36.441220,41.625434,47.368789,...,63.972071,88.148328,73.593792,65.886336,95.662420,92.085136,81.458887,48.457185,112.288493,200.038200
3,13.554817,11.230534,14.981784,15.615197,24.392023,42.991594,35.125547,39.789552,44.189902,34.470025,...,83.651565,69.619906,59.226047,81.627508,76.711621,131.543852,97.092892,100.843553,107.088481,130.023414
4,13.554817,11.230534,14.981784,15.615197,24.392023,29.283033,38.106378,34.939990,51.943817,58.943541,...,73.493085,86.777946,135.434475,114.259173,120.976459,75.845203,98.909781,42.119583,83.799838,116.464722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13.554817,11.230534,14.981784,15.615197,24.392023,28.497176,29.544771,20.124916,37.847109,39.166316,...,90.468530,83.230038,62.379074,87.665469,112.539723,81.632169,70.572127,72.957617,78.673018,97.840408
9996,13.554817,11.230534,14.981784,15.615197,24.392023,26.097270,31.488451,36.709498,66.368737,60.775803,...,86.285399,74.821101,89.231320,117.275085,84.668818,88.239529,98.892743,126.313039,102.089180,110.392288
9997,13.554817,11.230534,14.981784,15.615197,24.392023,28.516439,33.632517,40.767156,42.469752,76.910344,...,65.700190,104.977189,66.422375,85.786696,61.579470,79.960065,94.353418,68.566710,77.885622,104.562172
9998,13.554817,11.230534,14.981784,15.615197,24.392023,27.841891,33.910676,45.582615,46.004264,48.998074,...,70.624313,62.343703,58.698792,78.607691,94.419088,103.316571,92.063151,75.174135,83.288763,96.702104


In [36]:
display(pd.DataFrame(L.T))

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,1.810035,1.996299,1.706123,1.66287,1.214848,0.725211,0.435337,0.454004,0.774059,0.625132,...,0.387208,0.346969,0.069394,-0.049049,0.065856,0.036206,0.184657,0.270132,0.048970,0.157786
1,1.810035,1.996299,1.706123,1.66287,1.214848,1.005190,1.069911,0.903255,0.922719,1.304800,...,-0.043077,-0.503977,-0.126692,-0.442462,-0.307505,-0.253722,0.010619,-0.293527,-0.227459,0.987211
2,1.810035,1.996299,1.706123,1.66287,1.214848,0.945015,0.668410,0.808277,0.673682,0.542561,...,0.202174,-0.123190,0.053748,0.161430,-0.214861,-0.181222,-0.063496,0.451390,-0.340847,-0.699041
3,1.810035,1.996299,1.706123,1.66287,1.214848,0.646237,0.846476,0.720373,0.613897,0.860435,...,-0.066040,0.112780,0.270948,-0.052806,0.005912,-0.537849,-0.239065,-0.281499,-0.293431,-0.268247
4,1.810035,1.996299,1.706123,1.66287,1.214848,1.030233,0.765023,0.850346,0.452230,0.323945,...,0.063429,-0.107522,-0.556179,-0.389109,-0.449631,0.012797,-0.257605,0.591558,-0.048207,-0.158121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.810035,1.996299,1.706123,1.66287,1.214848,1.057437,1.019498,1.402019,0.768838,0.732708,...,-0.144382,-0.065777,0.219079,-0.124167,-0.377341,-0.060732,0.079968,0.042192,0.014924,0.016130
9996,1.810035,1.996299,1.706123,1.66287,1.214848,1.145411,0.955784,0.800942,0.207167,0.293333,...,-0.097040,0.040731,-0.138923,-0.415162,-0.092782,-0.138563,-0.257433,-0.506692,-0.245622,-0.104573
9997,1.810035,1.996299,1.706123,1.66287,1.214848,1.056761,0.889912,0.696101,0.653601,0.057884,...,0.175518,-0.297912,0.156275,-0.102503,0.225636,-0.040036,-0.210445,0.104264,0.024983,-0.050314
9998,1.810035,1.996299,1.706123,1.66287,1.214848,1.080700,0.881675,0.584451,0.573659,0.508744,...,0.103246,0.223168,0.279890,-0.015109,-0.201778,-0.296306,-0.185872,0.012264,-0.042089,0.027832


In [37]:
data

,date,Qgage,Qmodel,month,lambda,raw diff,Ar_res
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708,1.786596
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749,1.411201
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814,0.761985
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705,0.583921
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714,0.070037
...,...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934,-0.022115
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129,-0.024599
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849,0.261246
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763,-0.024250


## <font color = 'green'> Save Outputs </font>

Outputs saved as **.npy** files. The outputs are:

1. E.npy: stochastic residuals
2. L.npy: stochastic log ratios
3. Q.npy: stochastic streamflows

In [38]:
# save the results as .npy files
np.save('Q_baseline.npy',Q)
np.save('StochasticE_baseline.npy',E)
np.save('L_baseline.npy',L)

***

# <center> <font color = 'red'> Part 2: Diagnostic Figures </font> </center>

Here, we include code to generate T-year flood flows for the specified warming scenario.

## <font color = 'green'> Reloading in the data </font>

In [2]:
# functions developed by Ghazal:
import Func_Lib_forAbby as func

# general packages
import numpy as np #numpy is a scientific computing package
import pandas as pd #pandas is a data analysis package
import math #math contains math functions
import time #functions in this package can be used to understand how long the code is taking to run
import os #functions related to operating system

# plotting packages
import matplotlib.pyplot as plt #matplotlib.pyplot is used for plotting
import seaborn as sns #seaborn is a plotting package
plt.style.use('seaborn-white') #this is the specific style of plotting used

# statistical analysis packages
from scipy import stats #scipy.stats are functions related to statistical analysis
from scipy.stats import norm #functions for normally distributed random variable 

# time series analysis packages
import statsmodels as sm
from statsmodels.graphics.tsaplots import plot_acf #function to plot autocorrelation function (acf) for time series analysis
from statsmodels.graphics.tsaplots import plot_pacf #function to plot partial autocorrelation function (pacf) for time series analysis
#from statsmodels.tsa.arima_model import ARIMA #import function for ARIMA (time series analysis)
from statsmodels.tsa.arima.model import ARIMA

In [6]:
# read in the csv of streamflow data and assign to variable called "data"
data = pd.read_csv('SWM-data_cond-debias.csv') #columns are date, Qgage (observed), Qmodel (deterministic model), and diff (Qmodel - Qgage)

data['month'] = pd.DatetimeIndex(data['date']).month #create a DatetimeIndex column for month
data['date'] = pd.to_datetime(data['date']) #convert format of "date" column to Datetime (for time series analysis)

#convert units to m^3/s 
# mm/day * SHA area in mi2 * (1609.34^2 m^2/mi^2) * (10^-3 m/1 mm) * (1 day/24*3600 s)
area_mi2_SHA = 6665 #area for Shasta baisn in sq. mi. from Zach Brodeur
data['Qgage'] = data['Qgage']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
data['Qmodel'] = data['Qmodel']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)

# generating empirical log-ratio errors
data['lambda'] = np.log(data['Qmodel'] / data['Qgage']) #calculate the lambda = log(Qmodel/Qgage)
data['raw diff'] = data['Qmodel'] - data['Qgage']
data #display first five rows of data

,date,Qgage,Qmodel,month,lambda,raw diff
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714
...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763


In [7]:
#load in a "future" scenario into a new datafrae df_fut
df_fut = pd.read_csv('SWM-data_cond-debias_scenario-1.csv') #make sure this is one of the five we want, I just used a random one

#put the future scenario into the correct units of m3/s
df_fut['Qfut'] = df_fut['Qsim_cbias']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
df_fut['date'] = pd.to_datetime(df_fut['date'])
df_fut = df_fut[(df_fut['date'] > '1987-09-30') & (df_fut['date'] < '2018-10-01')]
df_fut = df_fut.reset_index()
df_fut['index']=np.arange(0, 11323)

#df_fut = df_fut.iloc[0:11323]

df_fut

,index,date,Qsim,Qsim_cbias,Qfut
0,0,1987-10-01,0.010082,0.422703,84.453473
1,1,1987-10-02,0.009904,0.421925,84.298002
2,2,1987-10-03,0.009714,0.421092,84.131500
3,3,1987-10-04,0.009538,0.420316,83.976600
4,4,1987-10-05,0.009348,0.419472,83.807879
...,...,...,...,...,...
11318,11318,2018-09-26,0.013803,0.392048,78.328773
11319,11319,2018-09-27,0.013377,0.390136,77.946769
11320,11320,2018-09-28,0.012988,0.388372,77.594314
11321,11321,2018-09-29,0.017419,0.407531,81.422180


In [3]:
Q = np.load('Q_baseline.npy')
m=100
n=100

In [4]:
pd.DataFrame(Q)

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,13.554817,11.230534,14.981784,15.615197,24.392023,39.726960,52.988130,51.933990,37.650044,43.614619,...,53.165617,55.084171,72.451372,81.321422,72.248276,74.090379,63.557573,58.086821,76.039594,84.917629
1,13.554817,11.230534,14.981784,15.615197,24.392023,30.025620,28.092256,33.139409,32.449148,22.103253,...,81.752544,128.999411,88.146602,120.520885,104.948489,99.009263,75.639922,102.063779,100.251621,37.049554
2,13.554817,11.230534,14.981784,15.615197,24.392023,31.887895,41.971683,36.441220,41.625434,47.368789,...,63.972071,88.148328,73.593792,65.886336,95.662420,92.085136,81.458887,48.457185,112.288493,200.038200
3,13.554817,11.230534,14.981784,15.615197,24.392023,42.991594,35.125547,39.789552,44.189902,34.470025,...,83.651565,69.619906,59.226047,81.627508,76.711621,131.543852,97.092892,100.843553,107.088481,130.023414
4,13.554817,11.230534,14.981784,15.615197,24.392023,29.283033,38.106378,34.939990,51.943817,58.943541,...,73.493085,86.777946,135.434475,114.259173,120.976459,75.845203,98.909781,42.119583,83.799838,116.464722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13.554817,11.230534,14.981784,15.615197,24.392023,28.497176,29.544771,20.124916,37.847109,39.166316,...,90.468530,83.230038,62.379074,87.665469,112.539723,81.632169,70.572127,72.957617,78.673018,97.840408
9996,13.554817,11.230534,14.981784,15.615197,24.392023,26.097270,31.488451,36.709498,66.368737,60.775803,...,86.285399,74.821101,89.231320,117.275085,84.668818,88.239529,98.892743,126.313039,102.089180,110.392288
9997,13.554817,11.230534,14.981784,15.615197,24.392023,28.516439,33.632517,40.767156,42.469752,76.910344,...,65.700190,104.977189,66.422375,85.786696,61.579470,79.960065,94.353418,68.566710,77.885622,104.562172
9998,13.554817,11.230534,14.981784,15.615197,24.392023,27.841891,33.910676,45.582615,46.004264,48.998074,...,70.624313,62.343703,58.698792,78.607691,94.419088,103.316571,92.063151,75.174135,83.288763,96.702104


## <font color = 'green'> Generating Figure Inputs </font>

In [8]:
def annualmax(Q,data,n):
    annual_max=np.array(pd.DataFrame(Q.T,index=data['date']).resample('Y').max())
    annualmaxsort = np.zeros(annual_max.shape)
    for i in range(0,n):
        annualmaxsort[:,i]=np.sort(annual_max[:,i])[::-1]
    
    annualmaxsort = pd.DataFrame(annualmaxsort)
    #annualmaxsort['rank']=(annualmaxsort.index) + 1
    #annualmaxsort['Exeedance']=annualmaxsort['rank']/(len(annualmaxsort)+1)
    #annualmaxsort['non_Exceedance']=1-annualmaxsort['Exeedance']
    #Data=data.set_index('date')
    #annualmaxobs=Data['Qgage'].resample('Y').max()
    #annualmaxmodel=Data['Qmodel'].resample('Y').max()
    #annualmaxsort['obs']=np.sort(annualmaxobs)[::-1]
    #annualmaxsort['model']=np.sort(annualmaxmodel)[::-1]
    amax=np.array(annualmaxsort)[:,0:n]
    return annualmaxsort
Annualmax = annualmax(Q, df_fut, m*n)

In [10]:
# calculating 50-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/50)
flood50Y0=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood50Y0 = np.array(flood50Y0)

In [27]:
# calculating 100-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/100)
flood100Y0=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood100Y0 = np.array(flood100Y0)

In [28]:
# calculating 500-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/500)
flood500Y0=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood500Y0 = np.array(flood500Y0)

In [48]:
np.save('flood50Y0.npy',flood50Y0)
np.save('flood100Y0.npy',flood100Y0)
np.save('flood500Y0.npy',flood500Y0)